# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex, 
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)

In [3]:
# additional CSS to consider. 
# TODO: Current into each exported file. 
#       This should be set once into the webpages directly.
#       With only the class/id definitions in the .md files.

debug_html = """
<style>
#wrap{ overflow:auto; }
#fig1{ background:yellow; width:100%; float:left; padding:5px;  }
#fig2{ background:red; width:50%; float:left; clear:left; padding:5px;  }
#fig3{ background:green; width:50%; float:left; padding:5px;   }
.macros{ background:yellow; visibility:visible;}
h1 {margin: 0 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
""" 
html = """
<style>
#wrap{ overflow:auto; }
#fig1{ width:100%; float:left; padding: 5px;  }
#fig2{ width:50%; float:left; clear:left; padding: 5px;  }
#fig3{ width:50%; float:left; padding: 5px;  }
.macros{ visibility:hidden; height:0px; }
h1 {margin: 0em 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
"""

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [4]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

# select only papers with matching author names and highlight authors
hl_list = [k[0] for k in mpia_authors]

candidates = []
for paperk in new_papers:
    hl_authors = highlight_authors_in_list(paperk['authors'], hl_list)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

Arxiv has 49 new papers today
          21 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates[:-1]):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - " +
                       "_" + paper['comments'] + "_")
        doc.highlight_authors_in_list(hl_list)

        full_md = doc.generate_markdown_text()
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/20 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2212.03247


/tmp/ipykernel_2018/4030337529.py:34: LatexWarning: 2212.03247 did not run properly
not a gzip file
  warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +


Retrieving document from  https://arxiv.org/e-print/2212.03252


extracting tarball to tmp_2212.03252...

...

extracting tarball to tmp_2212.03742...

 done.
Retrieving document from  https://arxiv.org/e-print/2212.03746


extracting tarball to tmp_2212.03746...

 done.
Retrieving document from  https://arxiv.org/e-print/2212.03837


/tmp/ipykernel_2018/4030337529.py:34: LatexWarning: 2212.03837 did not run properly
not a gzip file
  warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2212.03514-b31b1b.svg)](https://arxiv.org/abs/arXiv:2212.03514) | **The CARMENES search for exoplanets around M dwarfs. Variability on long  timescales as seen in chromospheric indicators**  |
|| B. Fuhrmeister, et al. -- incl., <mark>M. Kürster</mark> |
|*Appeared on*| *2022-12-08*|
|*Comments*| *Accepted to A&A, 16 pages, 12 figures*|
|**Abstract**| It is clearly established that the Sun has an 11-year cycle that is caused by its internal magnetic field. This cycle is also observed in a sample of M dwarfs. In the framework of exoplanet detection or atmospheric characterisation of exoplanets, the activity status of the host star plays a crucial role, and inactive states are preferable for such studies. This means that it is important to know the activity cycles of these stars. We study systematic long-term variability in a sample of 211 M dwarfs observed with CARMENES, the high-resolution optical and near-infrared spectrograph at Calar Alto Observatory. In an automatic search using time series of different activity indicators, we identified 26 stars with linear or quadratic trends or with potentially cyclic behaviour. Additionally, we performed an independent search in archival R$^{\prime}_{\rm HK}$ data collected from different instruments whose time baselines were usually much longer. These data are available for a subset of 186 of our sample stars. Our search revealed 22 cycle candidates in the data. We found that the percentage of stars showing long-term variations drops dramatically to the latest M dwarfs. Moreover, we found that the pseudo-equivalent width (pEW) of the H$\alpha$ and Ca ii infrared triplet more often triggers automatic detections of long-term variations than the TiO index, differential line width, chromatic index, or radial velocity. This is in line with our comparison of the median relative amplitudes of the different indicators. For stars that trigger our automatic detection, this leads to the highest amplitude variation in R$^{\prime}_{\rm HK}$, followed by pEW(H$\alpha$), pEW(Ca ii IRT), and the TiO index. |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2212.03563-b31b1b.svg)](https://arxiv.org/abs/arXiv:2212.03563) | **A high resolution study of near-IR diffuse interstellar bands, search  for small scale structure, time variability and stellar features**  |
|| J. V. Smoker, et al. -- incl., <mark>A. Müller</mark>, <mark>K. T. Smith</mark> |
|*Appeared on*| *2022-12-08*|
|*Comments*| *Accepted for publicaion in the main journal of Astronomy and Astrophysics on 25th November 2022*|
|**Abstract**| Diffuse interstellar bands comprise hundreds of absorption features in the ISM. Most DIBs are observed in the optical, but some are in the IR. We observed 76 early-type stars at R=50,000 and S/N ratios of several hundreds using CRIRES. We measure DIBs around 1318, 1527, 1561, 1565, 1567, 1574 and/or 1624 nm. We detect a total of 6 DIB features and 17 likely stellar features assisted by a CMFGEN model. We also measured the DIBs at 1318 and 1527 nm using X-shooter towards ten Ceph. variables with 3.2 < E(B-V) < 6.5 and 4 stars at low values of water vapour. Correlation coeffs. of 0.73-0.96 are found comparing NIRDIB eq. width vs. E(B-V) and with r > 0.8 when comparing the NIR and optical DIBs 5705, 5780, 6203, 6283 and 6269 A. The 5797 A DIB is less well correlated with the NIDIBs. The "C60+" DIB at 9632 A is not well correlated with the 1318 nm DIB. Partial correlation coefficients using E(B-V) as the covariate were also determined. For stars earlier than B2, the 1318 nm DIB is affected by an emission line on its blue wing, likely stellar in nature, although we cannot rule out interstellar/circumstellar origin for example caused by by a DIB in emission. The 1318 nm DIB has a red wing and is reasonably well fitted by two gaussians. Neither the component ratios nor separation are correlated with 5780/5797 or E(B-V). EW(1318 nm) correlates with HI with EW(1318 nm)/E(B-V) decreasing with f(H2). Five pairs of stars within 1 am show similar 1318 nm DIB profiles. Variation in 1318 nm is seen in HD 145501/145502 and HD 168607/168625 pairs. CRIRES data for 17 stars separated by 6-14 months and 2 X-shooter sightlines separated by 9.9 yr were analysed. No time-variability is detected in the 5780, 5797 A, 1318 nm or 1527 nm DIBs. Tentative time variation is observed in the C60+ DIBs at 9577 and 9632 A towards HD 183143 although very close to the noise level with confirmation required. |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2212.03257-b31b1b.svg)](https://arxiv.org/abs/arXiv:2212.03257) | **A General Framework for Removing Point Spread Function Additive  Systematics in Cosmological Weak Lensing Analysis**  |
|| <mark>Tianqing Zhang</mark>, et al. -- incl., <mark>Xiangchong Li</mark> |
|*Appeared on*| *2022-12-08*|
|*Comments*| *29 pages, 25 figures, submitted to MNRAS. Comments welcomed*|
|**Abstract**| Cosmological weak lensing measurements rely on a precise measurement of the shear two-point correlation function (2PCF) along with a deep understanding of systematics that affect it. In this work, we demonstrate a general framework for describing the impact of PSF systematics on the cosmic shear 2PCF, and mitigating its impact on cosmological analysis. Our framework can describe leakage and modeling error from all spin-2 quantities contributed by the PSF second and higher moments, rather than just the second moments. We interpret null tests using the HSC Year 3 (Y3) catalogs with this formalism, and find that leakage from the spin-2 combination of PSF fourth moments is the leading contributor to additive shear systematics, with total contamination that is an order of magnitude higher than that contributed by PSF second moments alone. We conducted a mock cosmic shear analysis for HSC Y3, and find that, if uncorrected, PSF systematics can bias the cosmological parameters $\Omega_m$ and $S_8$ by $\sim$0.3$\sigma$. The traditional second moment-based model can only correct for a 0.1$\sigma$ bias, leaving the contamination largely uncorrected. We conclude it is necessary to model both PSF second and fourth moment contamination for HSC Y3 cosmic shear analysis. We also reanalyze the HSC Y1 cosmic shear analysis with our updated systematics model, and identify a 0.07$\sigma$ bias on $\Omega_m$ when using the more restricted second moment model from the original analysis. We demonstrate how to self-consistently use the method in both real space and Fourier space, assess shear systematics in tomographic bins, and test for PSF model overfitting. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: '69117' keyword not found.</p> |

...


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2212.03746-b31b1b.svg)](https://arxiv.org/abs/arXiv:2212.03746) | **Models of Rotating Infall for the B335 Protostar**  |
|| Neal J. Evans II, et al. -- incl., <mark>Joel D. Green</mark>, <mark>Bo Zhao</mark>, <mark>Jeong-Eun Lee</mark> |
|*Appeared on*| *2022-12-08*|
|*Comments*| *22 pages, 20 figures, in press ApJ*|
|**Abstract**| Models of the protostellar source, B335, are developed using axisymmetric three-dimensional models to resolve conflicts found in one-dimensional models. The models are constrained by a large number of observations, including ALMA, Herschel, and Spitzer data. Observations of the protostellar source B335 with ALMA show red-shifted absorption against a central continuum source indicative of infall in the HCO$^+$ and HCN $J = 4\rightarrow 3$ transitions. The data are combined with a new estimate of the distance to provide strong constraints to three-dimensional radiative transfer models including a rotating, infalling envelope, outflow cavities, and a very small disk. The models favor ages since the initiation of collapse between $3 \times 10^4$ and $4 \times 10^4$ yr for both the continuum and the lines, resolving a conflict found in one-dimensional models. The models under-predict the continuum emission seen by ALMA, suggesting an additional component such as a pseudo-disk. The best-fitting model is used to convert variations in the 4.5 $\mu m$ flux in recent years into a model for a variation of a factor of 5-7 in luminosity over the last 8 years. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2212.03608-b31b1b.svg)](https://arxiv.org/abs/arXiv:2212.03608) | **A recipe for orbital eccentricity damping in the type-I regime for low  viscosity 2D-discs**  |
|| <mark>Gabriele Pichierri</mark>, <mark>Bertram Bitsch</mark>, Elena Lega |
|*Appeared on*| *2022-12-08*|
|*Comments*| *Submitted to A&A*|
|**Abstract**| It is known that gap opening depends on the disc's viscosity; however, eccentricity damping formulas have only been derived at high viscosities, ignoring partial gap opening. We aim at obtaining a simple formula to model $e$-damping of the type-I regime in low viscosity discs, where even small planets may start opening partial. We perform high resolution 2D locally isothermal hydrodynamical simulations of planets with varying masses on fixed orbits in discs with varying aspect ratios and viscosities. We determine the torque and power felt by the planet to derive migration and eccentricity damping timescales. We first find a lower limit to the gap depths below which vortices appear; this happens roughly at the transition between type-I and type-II regimes. For the simulations that remain stable, we obtain a fit to the observed gap depth in the limit of vanishing eccentricities that is similar to the one currently used in the literature but is accurate down to $\alpha=3.16\times 10^{-5}$. We record the $e$-damping efficiency as a function of the observed gap depth and $e$: when the planet has opened a deep enough gap, a linear trend is observed independently of $e$; at shallower gaps this linear trend is preserved at low $e$, while it deviates to more efficient damping when $e$ is comparable to the disc's scale height. Both trends can be understood on theoretical grounds and are reproduced by a simple fitting formula. Our combined fits yield a simple recipe to implement type-I $e$-damping in $N$-body for partial gap opening planets that is consistent with high-resolution 2D hydro-simulations. The typical error of the fit is of the order of a few percent, and lower than the error of type-I torque formulas widely used in the literature. This will allow a more self-consistent treatment of planet-disc interactions of the type-I regime for population synthesis models at low viscosities. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error Could not find figure DustTrapPlotGlobal</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2212.03728-b31b1b.svg)](https://arxiv.org/abs/arXiv:2212.03728) | **Adiabatic expansion for varying speed of light model**  |
|| <mark>Seokcheon Lee</mark> |
|*Appeared on*| *2022-12-08*|
|*Comments*| *5 pages, 3 tables*|
|**Abstract**| Recently, we proposed a varying speed of light (VSL) model to solve various late-time cosmological problems \cite{Lee:2020zts}. This model has one free parameter, $b$, to characterize the time variation of the speed of light as a function of a scale factor, $c = c_0a^{b/4}$. Time variations of various physical constants and quantities have different powers of scale factor as a function of $b$ to satisfy all known local physics laws, including special relativity, thermodynamics, and electromagnetic force. This model is based on the Robertson-Walker metric and satisfies the isotropic and homogeneous 3-space required by the cosmological principle. Adiabaticity is a necessary condition to keep homogeneity and isotropy because a net energy flux would falsify the isotropy if there is a preferential energy flow direction. It also might forge homogeneity if the outward (inward) flux is isotropic. Thus, any VSL model should also preserve an adiabatic expansion condition to be a viable model. Also, it provides an additional condition for constraining physical constants. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error invalid header</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2212.03247-b31b1b.svg)](https://arxiv.org/abs/arXiv:2212.03247) | **Ground-based Synoptic Studies of the Sun**  |
|| Sanjay Gosain, et al. -- incl., <mark>C. J. Henney</mark>, <mark>H. Lin</mark>, <mark>J. Zhang</mark> |
|*Appeared on*| *2022-12-08*|
|*Comments*| *10 pages, 5 figures, White paper submitted to Heliodecadal 2024, Category: Basic Research, Solar Physics. arXiv admin note: text overlap with arXiv:1903.06944*|
|**Abstract**| Ground-based synoptic solar observations provide critical contextual data used to model the large-scale state of the heliosphere. The next decade will see a combination of ground-based telescopes and space missions that will study our Sun's atmosphere microscopic processes with unprecedented detail. This white paper describes contextual observations from a ground-based network needed to fully exploit this new knowledge of the underlying physics that leads to the magnetic linkages between the heliosphere and the Sun. This combination of a better understanding of small-scale processes and the appropriate global context will enable a physics-based approach to Space Weather comparable to Terrestrial Weather forecasting. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error not a gzip file</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2212.03837-b31b1b.svg)](https://arxiv.org/abs/arXiv:2212.03837) | **Surfing in the phase space of spin-orbit coupling in binary asteroid  systems**  |
|| <mark>Mahdi Jafari Nadoushan</mark> |
|*Appeared on*| *2022-12-08*|
|*Comments*| *16 pages, 16 figures, accepted for publication in MNRAS*|
|**Abstract**| For a satellite with an irregular shape, which is the common shape among asteroids, the well-known spin-orbit resonance problem could be changed to a spin-orbit coupling problem since a decoupled model does not accurately capture the dynamics of the system. In this paper, having provided a definition for close binary asteroid systems, we explore the structure of the phase space in a classical Hamiltonian model for spin-orbit coupling in a binary system. To map out the geography of resonances analytically and the cartography of resonances numerically, we reformulate a fourth-order gravitational potential function, in Poincare variables, via Stokes coefficients. For a binary system with a near-circular orbit, isolating the Hamiltonian near each resonance yields the pendulum model. Analysis of the results shows the geographical information, including the location and width of resonances, is modified due to the prominent role of the semi-major axis in the spin-orbit coupling model but not structurally altered. However, this resulted in modified Chirikov criterion to predict onset of large-scale chaos. For a binary system with arbitrary closed orbit, we thoroughly surf in the phase space via cartography of resonances created by fast Lyapunov indicator (FLI) maps. The numerical study confirms the analytical results, provides insight into the spin-orbit coupling, and shows some bifurcations in the secondary resonances which can occur due to material transfer. Also, we take the (65803) Didymos binary asteroid as a case to show analytical and numerical results. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error not a gzip file</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2212.03514.md
    + _build/html/tmp_2212.03514/./J22330cycle_e.png
    + _build/html/tmp_2212.03514/./J10251rp_e.png
    + _build/html/tmp_2212.03514/./J10289volker_e.png
    + _build/html/tmp_2212.03514/./J06105cycle_e.png
exported in  _build/html/2212.03563.md
    + _build/html/tmp_2212.03563/./Plot_Different_Emission_Velocities.png
    + _build/html/tmp_2212.03563/./PlotSmallScaleStructure_HR6026_HD145501_Field.png
    + _build/html/tmp_2212.03563/./CRIRES_Spectra_Examples_PAGE01_Annotated.png


## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\tbd}[1]{{[TBD: {\bfseries #1}]}}$
$\newcommand{\comm}[2]{\textit{#1} \textbf{#2}}$
$\newcommand{\vsini}{v \sin{i}}$
$\newcommand{\hei}[0]{\ion{He}{i}}$
$\newcommand{\heir}[0]{\hei IR}$
$\newcommand{\itio}[0]{I_{\mathrm{TiO}}}$
$\newcommand{\kirb}[0]{\ion{K}{i} IR_{\rm blue}}$
$\newcommand{\kirr}[0]{\ion{K}{i} IR_{\rm red}}$
$\newcommand{\kopt}[0]{\ion{K}{i}  VIS}$
$\newcommand{\koptb}[0]{\ion{K}{i} VIS_{\rm blue}}$
$\newcommand{\koptr}[0]{\ion{K}{i} VIS_{\rm red}}$
$\newcommand{\cairta}[0]{\ion{Ca}{ii} IRT_{\rm 1}}$
$\newcommand{\cairtb}[0]{\ion{Ca}{ii} IRT_{\rm 2}}$
$\newcommand{\cairt}[0]{\ion{Ca}{ii} IRT}$
$\newcommand{\cahk}[0]{\ion{Ca}{ii}~H \& K }$
$\newcommand{\rpr}[0]{R^{\prime}_{\rm HK} }$</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{$\ensuremath$}{}$
$\newcommand{$\xspace$}{}$
$\newcommand{$\object$}[1]{\texttt{#1}}$
$\newcommand{$\farcs$}{{.}''}$
$\newcommand{$\farcm$}{{.}'}$
$\newcommand{$\arcsec$}{''}$
$\newcommand{$\arcmin$}{'}$
$\newcommand{$\ion$}[2]{#1#2}$
$\newcommand{$\textsc$}[1]{\textrm{#1}}$
$\newcommand{$\hl$}[1]{\textrm{#1}}$
$\newcommand{$\tbd$}[1]{{[TBD: {\bfseries #1}]}}$
$\newcommand{$\comm$}[2]{\textit{#1} \textbf{#2}}$
$\newcommand{$\vsini$}{v \sin{i}}$
$\newcommand{$\hei$}[0]{$\ion${He}{i}}$
$\newcommand{$\hei$r}[0]{$\hei$ IR}$
$\newcommand{$\itio$}[0]{I_{\mathrm{TiO}}}$
$\newcommand{$\kirb$}[0]{$\ion${K}{i} IR_{\rm blue}}$
$\newcommand{$\kirr$}[0]{$\ion${K}{i} IR_{\rm red}}$
$\newcommand{$\kopt$}[0]{$\ion${K}{i}  VIS}$
$\newcommand{$\kopt$b}[0]{$\ion${K}{i} VIS_{\rm blue}}$
$\newcommand{$\kopt$r}[0]{$\ion${K}{i} VIS_{\rm red}}$
$\newcommand{$\cairta$}[0]{$\ion${Ca}{ii} IRT_{\rm 1}}$
$\newcommand{$\cairtb$}[0]{$\ion${Ca}{ii} IRT_{\rm 2}}$
$\newcommand{$\cairt$}[0]{$\ion${Ca}{ii} IRT}$
$\newcommand{$\cahk$}[0]{$\ion${Ca}{ii}~H \& K }$
$\newcommand{$\rpr$}[0]{R^{\prime}_{\rm HK} }$</div>



<div id="title">

# The CARMENES search for exoplanets around M dwarfs: Variability on long timescales as seen in chromospheric indicators

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2212.03514-b31b1b.svg)](https://arxiv.org/abs/2212.03514)<mark>Appeared on: 2022-12-08</mark> - _Accepted to A&A, 16 pages, 12 figures_

</div>
<div id="authors">

B. Fuhrmeister, et al. -- incl., <mark><mark>M. Kürster</mark></mark>

</div>
<div id="abstract">

**Abstract:** It is clearly established that the Sun has an  11-year cycle that is caused by itsinternal magnetic field.  This cycle is also observed in a sample of M dwarfs. In the framework of exoplanet detection  or atmospheric characterisation of exoplanets, the activity status of the  host star plays a crucial role, and inactive states are preferable for such studies. This means that  it is important to know the activity cycles of these stars. We study systematic long-term variability  in a sample of 211 M dwarfs observed with CARMENES, the high-resolution optical and near-infrared  spectrograph at Calar Alto Observatory. In an automatic search using time series of different activity indicators, we  identified 26 stars with linear or quadratic trends or with potentially cyclic behaviour.   Additionally, we performed an independent search in archival$\rpr$data collected from different    instruments whose time    baselines were usually much longer. These data are available for a subset of 186 of our sample stars. Our search  revealed 22 cycle candidates in the data.  We found that  the percentage of stars showing long-term variations drops dramatically to the latest  M dwarfs.  Moreover, we found that the pseudo-equivalent width (pEW) of the  H$\alpha$and$\ion${Ca}{ii}infrared triplet   more often triggers automatic detections of long-term variations  than the TiO index,  differential line width,  chromatic index, or radial velocity. This is in line with  our comparison of the median relative amplitudes of the different indicators. For stars that trigger our automatic detection,  this leads to the highest amplitude variation in$\rpr$, followed by pEW(H$\alpha$), pEW($\cairt$), and the TiO index.

</div>

<div id="div_fig1">

<img src="tmp_2212.03514/./J22330cycle_e.png" alt="Fig5" width="100%"/>

**Figure 5. -** \label{fig:J22330} GLS and time series of J22330+093 / BD+08 4887. We
         show the GLS (\emph{left, top}) with pEW(H$\alpha$) in black,
         pEW($\cairta$) in blue, pEW($\cairtb$) in green, the TiO index in magenta,
         RV in yellow, dLw in grey, CRX in orange, and the window function in red.
         The dashed blue line indicates FAP=0.01 for the pEW($\cairta$) data.
         Additionally, we show the time series for pEW(H$\alpha$) (\emph{right, top}),
         for pEW($\cairta$) (\emph{left, bottom}), and the TiO index (\emph{right, bottom}).
         The dashed black lines indicate the best sine fit for each indicator.
 (*fig:J22330*)

</div>
<div id="div_fig2">

<img src="tmp_2212.03514/./J10251rp_e.png" alt="Fig11.1" width="50%"/><img src="tmp_2212.03514/./J10289volker_e.png" alt="Fig11.2" width="50%"/>

**Figure 11. -** \label{fig:rpalone1} GLS and time series of J10251$-$102 / BD$-$09 3070 (M1.0 V, \emph{left}) and
  J10289+008 / BD+01 2447 (M2.0 V, \emph{right}). In the GLS (\emph{top}), we show the power of the $\rpr$   data in cyan, the dashed cyan line marks the 0.0005 FAP level, and the red line
  is the power of the window function. In the time series (\emph{bottom}), we show the
  $\rpr$ values as cyan dots and the best-fit sine function as the dashed black line.
 (*fig:rpalone1*)

</div>
<div id="div_fig3">

<img src="tmp_2212.03514/./J06105cycle_e.png" alt="Fig8" width="100%"/>

**Figure 8. -** \label{fig:J06105} Same as in Fig. \ref{fig:J22330}, but for J06105$-$218 / HD 42581 A.
         (*fig:J06105*)

</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand$
$\newcommand{\commentAna}[1]{\textcolor{olivegreen}{#1}}$
$\newcommand{\commentRosine}[1]{\textcolor{orange}{#1}}$
$\newcommand$
$\newcommand{\commentNick}[1]{\textcolor{blue}{#1}}$
$\newcommand{\csixty}{C_{60}\xspace}$
$\newcommand{\cseventy}{C_{70}\xspace}$
$\newcommand{\csixtyplus}{C_{60}^{+}\xspace}$
$\newcommand{\andres}[1]{\textcolor{purple}{{Andres: #1}}}$
$\newcommand{\anna}[1]{\textcolor{orange}{{Anna: #1}}}$
$\newcommand{\meriem}[1]{\textcolor{green}{{Meriem: #1}}}$
$\newcommand{\keith}[1]{\textcolor{red}{{Keith: #1}}}$
$\newcommand{\henri}[1]{\textcolor{brown}{{HB: #1}}}$
$\newcommand{\thefigure}{A\@arabic\c@figure}$</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{$\ensuremath$}{}$
$\newcommand{$\xspace$}{}$
$\newcommand{$\object$}[1]{\texttt{#1}}$
$\newcommand{$\farcs$}{{.}''}$
$\newcommand{$\farcm$}{{.}'}$
$\newcommand{$\arcsec$}{''}$
$\newcommand{$\arcmin$}{'}$
$\newcommand{$\ion$}[2]{#1#2}$
$\newcommand{$\textsc$}[1]{\textrm{#1}}$
$\newcommand{$\hl$}[1]{\textrm{#1}}$
$\newcommand$
$\newcommand{$\commentAna$}[1]{\textcolor{olivegreen}{#1}}$
$\newcommand{$\commentRosine$}[1]{\textcolor{orange}{#1}}$
$\newcommand$
$\newcommand{$\commentNick$}[1]{\textcolor{blue}{#1}}$
$\newcommand{$\csixty$}{C_{60}$\xspace$}$
$\newcommand{$\cseventy$}{C_{70}$\xspace$}$
$\newcommand{$\csixty$plus}{C_{60}^{+}$\xspace$}$
$\newcommand{$\andres$}[1]{\textcolor{purple}{{Andres: #1}}}$
$\newcommand{$\anna$}[1]{\textcolor{orange}{{Anna: #1}}}$
$\newcommand{$\meriem$}[1]{\textcolor{green}{{Meriem: #1}}}$
$\newcommand{$\keith$}[1]{\textcolor{red}{{Keith: #1}}}$
$\newcommand{$\henri$}[1]{\textcolor{brown}{{HB: #1}}}$
$\newcommand{$\thefigure$}{A\@arabic\c@figure}$</div>



<div id="title">

# A high resolution study of near-IR diffuse interstellar bands, search for small scale structure, time variability and stellar features

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2212.03563-b31b1b.svg)](https://arxiv.org/abs/2212.03563)<mark>Appeared on: 2022-12-08</mark> - _Accepted for publicaion in the main journal of Astronomy and Astrophysics on 25th November 2022_

</div>
<div id="authors">

J. V. Smoker, et al. -- incl., <mark><mark>A. Müller</mark></mark>, <mark><mark>K. T. Smith</mark></mark>

</div>
<div id="abstract">

**Abstract:** The diffuse interstellar bands (DIBs) are a set of hundreds of unidentified absorption features that appear near ubiquitously throughout the interstellar medium. Most DIBs appear at optical wavelengths, but some are in the near-infrared.We aim to characterise near-infrared DIBs at high spectral resolving power towards multiple targets.We observed 76 early-type stars at a resolving power of 50,000 (velocity resolution$\sim$6 kms$^{-1}$) and signal to noise ratios of several hundreds using the CRyogenic high-resolution InfraRed Echelle Spectrograph (CRIRES). These data allow us to investigate the DIBs around 1318.1, 1527.4, 1561.1, 1565.1, 1567.0, 1574.4 and/or 1624.2 nm. We detect a total of six DIB features and 17 likely stellar features assisted by comparisons with a model spectrum computed with{\sc cmfgen}.Additionally, we measured equivalent widths of the DIBs at 1318.1 and 1527.4 nm using observations with X-shooter towards ten very highly reddened (3.2$< E(B-V) < $6.5) Cepheid variable stars, towards four stars observed at low values of precipitable water vapour and using other archive data.We measure correlations (correlation coefficient$r\sim$0.73--0.96, depending on the sub-sample used) between DIB equivalent width and reddening for the DIBs at1318.1, 1561.1, 1565.1 and 1567.0 nm. Comparing the near-infrared DIBs with 50 of the strongest optical DIBs, we find correlations$r$> 0.8 between the 1318, 1527, 1561, 1565 or 1567 nmand optical DIBs 5705, 5780, 6203, 6283 and 6269\AA. The 5797\AADIB is less well correlated with the near-infrared DIBs. The DIB at 9632.1\AA, (likely C$_{60}^+$), is not well correlated with the 1318.1 nm DIB. Partial correlation coefficients using$E(B-V)$as the covariate were also determined.For stars earlier than B2, the 1318.1 nm DIB is affected by an emission line on its blue wing that is likely stellar in nature, although we cannot rule out an interstellar or circumstellar origin for this line caused by for example a DIB in emission. The 1318.1 nm DIB also has an extended red wing. The line is reasonably well fitted by two gaussian components although neither the component ratios nor separation between them are obviously correlated with indicators of such as$\lambda\lambda$5780/5797 and reddening. EW(1318 nm) correlates with H{\sc i}with EW(1318 nm)/$E(B-V)$decreasing with f(H$_{2}$).Five pairs of stars within one arcmin of each other show very similar 1318.1 nm DIB profiles. Possible variation in the 1318.1 nm feature is seen between HD 145501 and HD 145502 (separated by 41 arcsec, equivalent to 7200 au) and HD 168607 and HD 168625 (separated by 67 arcsec, equivalent to 0.52 pc on the plane of the sky).Seventeen sightlines have repeat CRIRES observations separated by 6--14 months and two sightlines have repeat X-shooter observations separated by 9.9 years. No time-variability is detected in the 1318.1 nm DIB in the CRIRES data or the 5780.5, 5797.1 1318.1 and 1527.4 nm DIBs. Tentative time variation is observed in the "C$_{60}^+$" DIBs at 9577 and 9632\AAtowards HD 183143 although very close to the noise level and with confirmation required.The NIRDIBs observed occur more in more UV-irradiated regions than the 5797\AADIB allowing the study of heavily reddened sightlines. Future searches for time variability in DIBs require higher quality data and/or larger intervals between epochs.

</div>

<div id="div_fig1">

<img src="tmp_2212.03563/./Plot_Different_Emission_Velocities.png" alt="Fig2" width="100%"/>

**Figure 2. -** (a) Difference in absorption and emission line peak velocity around 1318 nm plotted against the stellar velocity. (b) Ditto for stellar velocity minus emission line velocity. (c) Emission line velocity vs stellar velocity, assuming the emission line has a rest wavelength of 1317.37 nm. Results with the same stellar velocities are from repeat observations of a given star. Purple points are from Gaia DR2 with Red points from Gaia DR3 for HD 160065 (*Emission_minus_Stellar_Velocity*)

</div>
<div id="div_fig2">

<img src="tmp_2212.03563/./PlotSmallScaleStructure_HR6026_HD145501_Field.png" alt="Fig8" width="100%"/>

**Figure 8. -** Spectra of the HD 145501 / HD 144502 field in the Ca {\sc ii} and K {\sc i} lines and 5780, 5797 \AA   and 1318.1 nm DIBs. We recall HD 145502 is a close binary. (*f_sss_HD_144501_HD_144502*)

</div>
<div id="div_fig3">

<img src="tmp_2212.03563/./CRIRES_Spectra_Examples_PAGE01_Annotated.png" alt="Fig13" width="100%"/>

**Figure 13. -** Normalised CRIRES spectra showing examples of all stellar and interstellar features observed. The ordinate is the normalised flux plus an offset with the abscissa being the wavelength in nm. Stellar lines: Red stars: 1315.4 nm, Green: 1316.8 nm, Cyan: 1316.5 nm (EmL), Purple: 1317.3 nm (EmL, uncertain), Light Green: 1529.9 nm, Yellow: 1566.1 nm (EmL), Gold: 1562.4, 1563.0, 1563.5 nm), Navy: 1633.8; Black: 1610.9 nm; Brown: 1555.7, 1616.7, 1625.3, 1626.1, 1634.2 nm (EmL). DIBs: Blue squares: 1318.1 nm, Green: 1527.1 nm, Cyan: 1561.1 nm, Purple: 1565.1 nm, Pink: 1567.0 nm. Not shown: broad and weak feature around 1564.4 nm. (*f_CRIRES_exaample_spectra_1*)

</div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")
[ print('\t', k) for k in res ];

129  publications in the last 7 days.
	 _build/html/2212.03563.md
	 _build/html/2212.03514.md
	 _build/html/2212.02667.md
	 _build/html/2212.02627.md
	 _build/html/2212.02235.md
	 _build/html/2212.01707.md
	 _build/html/2212.01526.md
	 _build/html/2212.01397.md
	 _build/html/2212.01291.md
	 _build/html/2212.01281.md
	 _build/html/2212.00812.md
	 _build/html/2212.00811.md
	 _build/html/2212.00806.md
	 _build/html/2212.00047.md
	 _build/html/2212.00032.md
	 _build/html/2211.16996.md
	 _build/html/2211.16510.md
	 _build/html/2211.16123.md
	 _build/html/2211.15816.md
	 _build/html/2211.15698.md
	 _build/html/2211.15576.md
	 _build/html/2211.15434.md
	 _build/html/2211.15288.md
	 _build/html/2211.15224.md
	 _build/html/2211.14543.md
	 _build/html/2211.14330.md
	 _build/html/2211.14322.md
	 _build/html/2211.14132.md
	 _build/html/2211.13811.md
	 _build/html/2211.13426.md
	 _build/html/2211.13318.md
	 _build/html/2211.13146.md
	 _build/html/2211.12613.md
	 _build/html/2211.11767.md
	 _build/h

In [11]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""<section class="carousel" aria-label="Gallery">""",
                """  <ol class="carousel__viewport">""",
    ]
    for k in range(1, npub + 1):
        prev_ = k - 1
        next_ = k + 1
        if prev_ <= 0:
            prev_ = npub
        if next_ > npub:
            next_ = 1
        text  = f"""    <li id="carousel__slide{k}" tabindex="0" class="carousel__slide">\n"""
        text += f"""       <div class="carousel__snapper">\n"""
        text += f"""         <a href="#carousel__slide{prev_}" class="carousel__prev">Go to previous slide</a>\n"""
        text += f"""         <a href="#carousel__slide{next_}" class="carousel__next">Go to next slide</a>\n"""
        text += f"""         <div id="slide{k}_content" class="md_view" >Content {k}</div>\n"""
        text += f"""       </div>\n"""
        text += f"""    </li>"""
        carousel.append(text)

    carousel.extend([
        """  </ol>""",
        """  <aside class="carousel__navigation">""",
        """    <ol class="carousel__navigation-list">"""])

    for k in range(1, npub + 1):
        text  = f"""      <li class="carousel__navigation-item">\n"""
        text += f"""        <a href="#carousel__slide{k}" class="carousel__navigation-button">Go to {k}</a>\n"""
        text += f"""      </li>"""
        carousel.append(text)
    carousel.extend(["""    </ol>""", """  </aside>""", """</section>"""])

    return '\n'.join(carousel)

In [12]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}_content"' for k in range(1, npub + 1)])

script = f"""
const docs = [{docs}]

const slides = [{slides}]
""" + """
async function run() {
    for (let i = 0; i < docs.length; i++) {
        let file = await fetch(docs[i]);
        let text = await file.text()
        document.getElementById(slides[i]).innerHTML =
            marked.parse(text);
    }
    hljs.highlightAll();
}
run()
"""

page = f"""<!doctype html>
<html lang="en">

<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <!-- Bootstrap CSS -->
  <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.0.2/dist/css/bootstrap.min.css" rel="stylesheet"
   integrity="sha384-EVSTQN3/azprG1Anm3QDgpJLIm9Nao0Yz1ztcQTwFspd3yD65VohhpuuCOmLASjC" crossorigin="anonymous">
  <!-- highlight.js CSS -->
  <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.1.0/styles/default.min.css">
  <!-- Mathjax 3 -->
  <script type="text/javascript" id="MathJax-config" src="mathjax_config.js"> </script>
  <script type="text/javascript" id="MathJax-script" async 
    src="https://cdn.jsdelivr.net/npm/mathjax@3/es5/tex-mml-chtml.js">
  </script>
  <link rel="stylesheet" href="index_carousel.css">
  <link rel="icon" type="image/x-icon" href="https://www.mpia.de/assets/touch-icon-32x32-a66937bcebc4e8894ebff1f41a366c7c7220fd97a38869ee0f2db65a9f59b6c1.png">
  <title>MPIA Arxiv on deck!</title>
</head>

<body>
  <div id="header"> <img src="header_banner.png" width="100%"></div>
  <div id="suptitle"> 7-day archives </div>
  <div id="info">
    <img src="https://pngimg.com/uploads/github/github_PNG58.png" height=30rem></img>
    <a href=https://github.com/mpi-astronomy/arxiv_display style="color:black;">github/mpi-astronomy/arxiv_display</a> 
  </div>
  {carousel:s}
</body>

<!-- Render Markdown -->

<body>
  <!-- highlight.js: https://highlightjs.org/download/ -->
  <script src="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.5.0/highlight.min.js"></script>
  <!-- marked.js -->
  <script src="https://cdn.jsdelivr.net/npm/marked/marked.min.js"></script>
  <script>{script:s}</script>
</body>
</html>
"""
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

# Debugging papers